In [ ]:
import numpy as np 
import os
%matplotlib inline
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch import Tensor
import torch
import torchvision

class Object(object): 
    pass

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device

In [ ]:
orig = Object()

In [ ]:
orig.model = models.resnet50(pretrained=True).to(device)

In [ ]:
orig.model

In [ ]:
orig.model.fc = nn.Sequential()
_ = orig.model.requires_grad_(False)

# Load Images

In [ ]:
input_path = "../input/walk-or-run"

In [ ]:
tr = Object()

In [ ]:
tr.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, .224, .225])
tr.transforms_train = transforms.Compose([
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    tr.normalize
])
tr.transforms_valid = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    tr.normalize
])

In [ ]:
tr.train = datasets.ImageFolder(input_path + '/walk_or_run_train/train', transform=tr.transforms_train)
tr.valid = datasets.ImageFolder(input_path + '/walk_or_run_test/test', transform=tr.transforms_valid)

In [ ]:
len(tr.train.samples)

In [ ]:
tr.train_loader = torch.utils.data.DataLoader(tr.train, batch_size=len(tr.train.samples))
tr.valid_loader = torch.utils.data.DataLoader(tr.valid, batch_size=len(tr.valid.samples))

In [ ]:
tr.train_loader.dataset.classes

In [ ]:
x, y = next(tr.train_loader.__iter__())
print(x.shape, y.shape)
tr.X_train = x
tr.train_embeddings = orig.model(x.to(device))
tr.train_y = y.to(device)

In [ ]:
x, y = next(tr.valid_loader.__iter__())
print(x.shape, y.shape)
tr.X_valid = x
tr.valid_embeddings = orig.model(x.to(device))
tr.valid_y = y.to(device)

In [ ]:
tr.valid_embeddings.shape

In [ ]:
tr.valid_embeddings[0]

In [ ]:
def train_embeddings(model, criterion, opt, epochs, train_emb, valid_emd, train_y, valid_y):
    for epoch in range(epochs):
        print(f'Epoch {epoch+1} of {epochs}')
        print('******************************')
        model.train()
        running_loss = 0
        running_correct = 0
        
        x = train_emb
        y = train_y
        outputs = model(x)
        loss = criterion(outputs, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * x.size(0)
        running_correct += torch.sum(preds==y.data)
        epoch_loss = running_loss / train_emb.shape[0]
        epoch_acc = running_correct.double() / train_emb.shape[0]
        print('Training loss: {:.4f}, accuracy: {:.4f}'.format(epoch_loss, epoch_acc))
        
        model.eval()
        running_loss = 0
        running_correct = 0
        x = valid_emd
        y = valid_y
        outputs = model(x)
        loss = criterion(outputs, y)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * x.size(0)
        running_correct += torch.sum(preds==y.data)
        epoch_loss = running_loss / valid_emd.shape[0]
        epoch_acc = running_correct.double() / valid_emd.shape[0]
        print('Validation loss: {:.4f}, accuracy: {:.4f}'.format(epoch_loss, epoch_acc))
        

In [ ]:
tr.train_y.shape

In [ ]:
tr.criterion = nn.CrossEntropyLoss()

tr.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
).to(device)

tr.optimizer = optim.SGD(tr.fc.parameters(), lr=0.01, momentum=0.9, nesterov=True)

_ = train_embeddings(
    tr.fc, 
    tr.criterion, 
    tr.optimizer, 
    80, 
    tr.train_embeddings, 
    tr.valid_embeddings,
    tr.train_y, 
    tr.valid_y)

# Make Predictions

In [ ]:
orig.model.fc = tr.fc

In [ ]:
tr.test_images = [
    input_path + '/walk_or_run_test/test/run/run_78b39d88.png',
    input_path + '/walk_or_run_test/test/run/run_365fa2e5.png',
    input_path + '/walk_or_run_test/test/run/run_603ac08a.png',
    input_path + '/walk_or_run_test/test/walk/walk_9e646bbc.png',
    input_path + '/walk_or_run_test/test/walk/walk_443d602c.png',
]

In [ ]:
from PIL import Image
tr.img_list = [Image.open(img_path).convert("RGB") for img_path in tr.test_images]

In [ ]:
tr.test_batch = torch.stack([
    tr.transforms_valid(img).to(device) for img in tr.img_list])

In [ ]:
tr.logits = orig.model(tr.test_batch)
tr.logits

In [ ]:
from torch.nn import functional as F
tr.proba = F.softmax(tr.logits, dim=1).cpu().data.numpy()
tr.proba

In [ ]:
tr.train_loader.dataset.classes

In [ ]:
tr.fig, tr.axs = plt.subplots(1, len(tr.img_list), figsize=(20, 5))
for i, img in enumerate(tr.img_list):
    ax = tr.axs[i]
    ax.axis('off')
    
    ax.set_title("{:.0f}% {}, {:.0f}% {}"
                 .format(100 * tr.proba[i,0], 
                         tr.train_loader.dataset.classes[0],
                         100 * tr.proba[i,1],
                         tr.train_loader.dataset.classes[1]))
    
    ax.imshow(img)

# Save Model for Redis AI

In [ ]:
import ml2rt

This will evaluate and convert model into a special TorchScript format. This format is universal and is longer tied to Python. So, we can load it inside Redis.

In [ ]:
orig.model.eval()
tr.nn_script = torch.jit.trace(orig.model, tr.X_valid.to(device))

Note: we can use TorchScript model just like any other PyTorch model. Now we can save the model:

In [ ]:
tr.nn_script.eval()
ml2rt.save_torch(tr.nn_script, 'model.pt')

In [ ]:
import redisai

In [ ]:
tr.redis = redisai.Client()

In [ ]:
tr.redis.loadbackend('TORCH', 'redisai_torch/redisai_torch.so')

In [ ]:
tr.loaded_model = ml2rt.load_model('model.pt')

In [ ]:
tr.redis.modelset(
    "walk_or_run", 
    redisai.Backend.torch,
    redisai.Device.cpu,
    tr.loaded_model)

In [ ]:
!ls ../input/walk-or-run/walk_or_run_test/test/walk/

In [ ]:
tr.img = Image \
    .open(input_path + '/walk_or_run_test/test/walk/walk_9d193f21.png') \
    .convert("RGB")
tr.img

Our model expects images in a certain format:
   * 224 x 224
   * Colors re-scaled the mean and std dev:
     * mean=[0.485, 0.456, 0.406]
     * std=[0.229, .224, .225]
   * The image must also be a part of an array, even if it's just one image. We will insert another dimension as the beginning to achieve this.

In [ ]:
tr.img_rescaled = tr.transforms_valid(tr.img)
tr.img_rescaled = np.expand_dims(tr.img_rescaled, axis=0)
tr.img_rescaled.shape

Send the image to Redis AI:

In [ ]:
tr.redis.tensorset('input', tr.img_rescaled)

In [ ]:
tr.redis.modelrun('walk_or_run', 'input', 'pred')

In [ ]:
tr.redis.tensorget('pred')

In [ ]:
torch.softmax(torch.Tensor(tr.redis.tensorget('pred')), dim=1)

classes = 'run', 'walk'